In [11]:
import pymysql
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

user='1111'
db=pymysql.connect(
    user='root',
    passwd='111111',
    host='localhost',
    db='instagram',
    charset='utf8'
)

cursor=db.cursor(pymysql.cursors.DictCursor)

sql="select id,group_concat(following_id) as follow from following group by id;"
cursor.execute(sql)
result=cursor.fetchall() #DB에서 user_post 받아오기

user_post_df=pd.DataFrame(result) #받아 온 값을 dataframe형식으로 변환 
user_post_df.head(1)

,id,follow
0,1111,"3333,5555"


In [12]:
user_post_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7 non-null      object
 1   follow  7 non-null      object
dtypes: object(2)
memory usage: 240.0+ bytes


In [13]:
print(user_post_df['follow'])

0         3333,5555
1         1111,6666
2    1111,2222,4444
3    2222,5555,6666
4              3333
5         2222,4444
6              2222
Name: follow, dtype: object


In [14]:
user_post_df['follow']=user_post_df['follow'].str.split(',') # ','로 나눔
print(user_post_df['follow'])

0          [3333, 5555]
1          [1111, 6666]
2    [1111, 2222, 4444]
3    [2222, 5555, 6666]
4                [3333]
5          [2222, 4444]
6                [2222]
Name: follow, dtype: object


In [15]:
user_post_df[['follow']][:1]

,follow
0,"[3333, 5555]"


In [16]:
# countvectorizer? 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩한 벡터를 만듬
# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.
user_post_df['follow_literal']=user_post_df['follow'].apply(lambda x:(' ').join(x))
print(user_post_df['follow_literal'])

#follow 문자열을 Count 백터화(나눈 문자의 출현 빈도를 세어 벡터(집함)로 변환)
count_vect=CountVectorizer(min_df=0,ngram_range=(1,2))#min_df = 최소 비교문자 , ngram_range = 
follow_mat=count_vect.fit_transform(user_post_df['follow_literal'])
print(follow_mat.shape)

0         3333 5555
1         1111 6666
2    1111 2222 4444
3    2222 5555 6666
4              3333
5         2222 4444
6              2222
Name: follow_literal, dtype: object
(7, 12)


In [17]:
#각 id에 따른 팔로우별 cosine유사도 추출(백터 사이의 각도만으로 유사도를 추출)
follow_sim=cosine_similarity(follow_mat,follow_mat)
print(follow_sim.shape)

(7, 7)


In [18]:
follow_sim_sorted_ind=follow_sim.argsort()[:,::-1]#index로 정렬 (::-1은 전체를 역순으로 조회)
print(follow_sim_sorted_ind)

[[0 4 3 6 5 2 1]
 [1 3 2 6 5 4 0]
 [2 5 6 1 3 4 0]
 [3 6 5 1 0 2 4]
 [4 0 6 5 3 2 1]
 [5 2 6 3 4 1 0]
 [6 5 3 2 4 1 0]]


In [19]:
#유사도가 높은 id를 찾아 반환
def find_sim_follow(df,sorted_ind,id):
    user_id=df[df['id']==id]
    id_index=user_id.index.values
    
    result_ind=[]
    user_id_follow=user_id['follow']
    for x in df['id']:
        if x in user_id_follow[0]:
            result_ind.append(df[df['id']==x].index.values)
        else:
            continue
    
    result_ind=np.array(result_ind).reshape(-1).tolist()
    similar_indexes=sorted_ind[id_index,:]
    similar_indexes=similar_indexes.reshape(-1)#해당 열(index)만 따로 저장
    similar_indexes = similar_indexes[similar_indexes != id_index]
    print(similar_indexes)
    similar_df=df.iloc[similar_indexes]#해당 index에 대한 아이디를 저장
    similar_df=similar_df.drop(result_ind)
    count_list=[]
    user_id_list=user_id['follow'].tolist()
    similar_df_list=similar_df['follow'].tolist()
    i=0
    #함께아는 친구 계산
    for lenght in range(len(similar_df_list)):#5번(추천된 5명의 count)
        count=0
        for x in similar_df_list[i]:#추천된 사람을 한 명씩 호출해 그 사람이 팔로우 한 사람을 찾음
            for y in user_id_list[0]:#자신이 팔로우한 사람
                if x in y:#자신이 팔로우한 사람과 추천된 사람의 팔로우를 비교
                    count=count+1
        i=i+1
        count_list.append(count)
                    
    similar_df['together']=count_list
    return similar_df

In [20]:
similar_id=find_sim_follow(user_post_df,follow_sim_sorted_ind,user)
similar_id[['id','together']]
print(similar_id[['id','together']])
similar_id=json.dumps(dict(np.array(similar_id[['id','together']]).tolist()))
print(similar_id)

[4 3 6 5 2 1]
     id  together
3  4444         1
6  7777         0
5  6666         0
1  2222         0
{"4444": 1, "7777": 0, "6666": 0, "2222": 0}
